In [16]:
import os
import h5py
from datetime import datetime
import pytz
import csv
import numpy as np
import pandas as pd
from scipy.signal import medfilt
from  scipy import ndimage
import matplotlib.pyplot as plt
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from scipy.optimize import curve_fit

def openFile():
    file = 'CSVFiles/csv_all_files.csv'
    file = open(file, "r")
    df = pd.read_csv(file)
    return df

def search_by_dataset_name(df, dataset):
    df = df[df['DatasetName'].str.contains(str(dataset))==True]
    if(len(df) == 1):
        filename = df['DatasetName'].values[0]
        parentGroup = df['GroupName'].values[0]
        H5File = df['HDF5FileName'].values[0]
        file = h5py.File("H5Files"+"/"+str(H5File), 'r')
        parentGroup = file[parentGroup]
        return get_dataset(file, parentGroup)
    else:
        return df

def get_dataset(h5File, parentGroup):
    dataset = {'DatasetName' : h5File.name}
    for file in parentGroup.keys() :
        values = parentGroup[file].name.split("/")
        val = list(parentGroup[file])
        if len(val) == 1 :
            dataset.update({values[len(values)-1] : val[0]})
        else :
            dataset.update({parentGroup.name+"/"+str(values[len(values)-1]) : list(val)})
    return dataset

def getImage(dataset):
    data = dataset["/AwakeEventData/XMPP-STREAK/StreakImage/streakImageData"]
    height = dataset["streakImageHeight"]
    width = dataset["streakImageWidth"]
    
    data = np.reshape(data, (height, width))
    data = medfilt(data)
    fig = plt.figure(figsize=(width/100,height/100))
    ax = fig.add_subplot(111)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    plt.imshow(data)
    plt.savefig('PNGFiles/DataAnalysis/Fig1.png')
    plt.close()
    
    plt.imshow(data, cmap=plt.cm.gray)
    plt.savefig('PNGFiles/DataAnalysis/Fig2.png')
    plt.close()
    
    rotate_data = ndimage.rotate(data, 90)
    plt.imshow(rotate_data, cmap=plt.cm.gray)
    plt.savefig('PNGFiles/DataAnalysis/Fig3.png')
    plt.close()
    
    blurred_data = ndimage.gaussian_filter(data, sigma=3)
    plt.imshow(blurred_data)
    plt.savefig('PNGFiles/DataAnalysis/Fig4.png')
    plt.close()
    
    plt.show(data)
    plt.savefig('PNGFiles/DataAnalysis/Fig5.png')
    plt.close()
    
    showImages()
    
def func(x, a, b, c):
    return a * np.exp(-b * x) + c

def showImages():
    images = []
    for img_path in glob.glob('PNGFiles/DataAnalysis/*.png'):
        images.append(mpimg.imread(img_path))

    plt.figure(figsize=(60,30))
    columns = 2
    val=0
    #title = ["NORMAL IMAGE", "IMAGE IN GRAYSCALE", "IMAGE ROTATED BY 90 DEGREES", "DENOISED IMAGE THAT WAS BLURRED"]
    for i, image in enumerate(images):
        plt.subplot(len(images) / columns + 1, columns, i + 1)
        #plt.title(title[val], fontsize=30)
        plt.imshow(image)
        val=val+1

df = openFile()
val = search_by_dataset_name(df, "/AwakeEventData/XMPP-STREAK/StreakImage/streakImageData")
getImage(val)



ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()